In [40]:
import numpy as np
import pandas as pd
import os.path

DSET_ROOT = "/Users/ever/Documents/HackathononKIBots/WorkTeam1/datasets/"
#DSET_ROOT = "https://raw.githubusercontent.com/gywn/WorkTeam1/master/datasets/all_nodes.json"

def load_est_data(est_data_path = DSET_ROOT):
    json_path=os.path.join(est_data_path,"all_nodes.json")
    return pd.read_json(json_path)
def load_station_data(est_data_path = DSET_ROOT):
    json_path=os.path.join(est_data_path,"haltestellen.json")
    return pd.read_json(json_path)
establishments = load_est_data()
stations = load_station_data() 
stations = stations.transpose()
stations = stations.reset_index(drop=True)
stations["lat"]=""
stations["lon"]=""
stations.id = pd.to_numeric(stations.id)
stations = stations.sort_values(by =["id"])


In [41]:
establishments['subtype'] = ""
tags = establishments.tags
types = establishments.type
type_df=establishments['type'].str[0:].str.split(':', expand=True)

In [43]:
establishments[0:3]
amenities_subtypes = establishments[establishments.type == 'amenity'].subtype.unique()
amenities_subtypes = pd.DataFrame(amenities_subtypes)
amenities_subtypes.to_csv("amenity_subtypes.csv")

tourism_subtypes = establishments[establishments.type == 'tourism'].subtype.unique()
tourism_subtypes = pd.DataFrame(tourism_subtypes)
tourism_subtypes.to_csv("tourism_subtypes.csv")

leisure_subtypes = establishments[establishments.type == 'leisure'].subtype.unique()
leisure_subtypes = pd.DataFrame(leisure_subtypes)
leisure_subtypes.to_csv("leisure_subtypes.csv")

shop_subtypes = establishments[establishments.type == 'shop'].subtype.unique()
shop_subtypes = pd.DataFrame(shop_subtypes)
shop_subtypes.to_csv("shop_subtypes.csv")

In [32]:
types = establishments.type.unique()
amenities_subtypes = establishments[establishments.type == 'amenity'].subtype.unique()
subtypes = pd.DataFrame(subtypes)
subtypes.to_csv("unique_subtypes.csv")

NameError: name 'subtypes' is not defined

In [44]:
def get_station_establishments(establishments,station_lat,station_lon,square_size = 0.0005):
    lat_low = station_lat - square_size
    lat_high = station_lat + square_size
    lon_low = station_lon - square_size
    lon_high = station_lon + square_size

    near_establishments = establishments[(establishments.lat<lat_high) &
                                         (establishments.lat>lat_low) &
                                         (establishments.lon<lon_high) &
                                         (establishments.lat>lon_low)]
    return near_establishments

station_stablishments=get_station_establishments(establishments,49.544802,8.670092,square_size=0.0002)


In [45]:
for ii in range(stations.shape[0]):
    stops1 = stations.stops[ii]
    stops1 = stops1[0]
    lat = stops1['lat']
    lon = stops1['lon']
    stations.lat[ii]=lat
    stations.lon[ii]=lon

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [46]:
stations_lat = stations['lat']
stations_lat.to_csv('../datasets/closest_stations/stations_lat.csv')
stations_lat = stations_lat.values

establishments_lat = establishments['lat']
establishments_lat.to_csv('../datasets/closest_stations/establishments_lat.csv')
establishments_lat = establishments_lat.values

stations_lon = stations['lon']
stations_lon.to_csv('../datasets/closest_stations/stations_lon.csv')
stations_lon = stations_lon.values

establishments_lon = establishments['lon']
establishments_lon.to_csv('../datasets/closest_stations/establishments_lon.csv')
establishments_lon = establishments_lon.values

In [47]:
lat_dist = np.subtract.outer(establishments_lat, stations_lat)
lon_dist = np.subtract.outer(establishments_lon, stations_lon)

In [48]:
#dist_matrix = np.sqrt([np.abs(lon_dist) + np.abs(lat_dist)])
dist_matrix = np.power([lon_dist**2 + lat_dist**2], 0.5 )
dist_matrix = np.squeeze(dist_matrix)
print(dist_matrix.shape)

closest_station = np.argmin( dist_matrix , 1)
print(closest_station.shape)

(39512, 931)
(39512,)


In [49]:
np.savetxt("../datasets/closest_stations/establishments_closest_station.csv", closest_station, delimiter=",")